In [24]:
import quandl
import pandas as pd
import seaborn as sns
import yfinance as yf 
import pandas as pd
from matplotlib import pylab as plt
import numpy as np
from datetime import datetime
import math
import quandl
import seaborn as sns
import sys
import pandas_datareader.data as web
from statsmodels.graphics.tsaplots import plot_acf
import datetime as dt
from dateutil.relativedelta import relativedelta

In [25]:
def Z_score(data):
    # not including the current observation
    std = data.iloc[:-1].std()
    mean = data.iloc[:-1].mean()
    Z = (data[-1:] - mean)/std
    return Z

In [26]:
def color_negative_red(value):
    """
      Colors elements in a dateframe
      green if positive and red if
      negative. Does not color NaN
      values.
      """

    if value < 0:
        color = 'red'
    elif value > 0:
        color = 'green'
    else:
        color = 'black'

    return 'color: %s' % color

# Sentiment analysis (Futures only)
- sp500
- gold
- dollar
- Bitcoin (Not option data)


Z score for net positions (1 and 3 years) and change in position (1 month and 3 months).

Append Relative Carry as well?



In [27]:
#data import

gold_tmp = quandl.get("CFTC/088691_F_L_ALL", authtoken="6rNSyTM5R4g7_B7XSyQW")
sp500_tmp = quandl.get("CFTC/13874A_F_L_ALL_NT", authtoken="6rNSyTM5R4g7_B7XSyQW") #futures only
NQ_tmp =quandl.get("CFTC/20974P_F_L_ALL", authtoken="6rNSyTM5R4g7_B7XSyQW")  #futures only
dxy_tmp = quandl.get("CFTC/098662_F_L_ALL", authtoken="6rNSyTM5R4g7_B7XSyQW")
bitcoin_tmp = quandl.get("CFTC/133741_F_L_ALL", authtoken="6rNSyTM5R4g7_B7XSyQW")
bonds_tmp = quandl.get("CFTC/020601_FO_L_ALL", authtoken="6rNSyTM5R4g7_B7XSyQW")
crude_tmp = quandl.get("CFTC/067651_F_L_ALL", authtoken="6rNSyTM5R4g7_B7XSyQW")



In [28]:

asset_names = ['SP500','NASDAQ','Bonds','Gold','Crude Oil', 'DXY', 'Bitcoin'  ]
assets = [sp500_tmp,NQ_tmp,bonds_tmp,gold_tmp, crude_tmp,dxy_tmp, bitcoin_tmp   ]

In [44]:
for asset in assets:
    ## BASED ON KANG ET AL
    try:
        asset['Net Position'] = asset['Noncommercial Long'] - asset['Noncommercial Short']
        asset['scld NP'] = asset['Net Position']/ asset['Open Interest']
    except:
        asset['Net Position'] = asset['Non Commercial Longs'] - asset['Non Commercial Shorts']
        asset['scld NP'] = asset['Net Position']
    
    asset['1 Yr Z Score'] = asset['scld NP'].rolling(52).apply(lambda x: Z_score(x),raw=False)
    asset['3 Yr Z Score'] = asset['scld NP'].rolling(3 * 52).apply(lambda x: Z_score(x),raw=False) 
    asset['5 Yr Z Score'] = asset['scld NP'].rolling(5 * 52).apply(lambda x: Z_score(x),raw=False) 
    asset['1 Month Change'] = asset['1 Yr Z Score'].diff(4)
    asset['3 Month Change'] = asset['1 Yr Z Score'].diff(4*3)

In [45]:
TOTAL_ASSETS = None

for asset in assets:
    
    TOTAL_ASSETS = pd.concat([TOTAL_ASSETS, asset.tail(1)], axis = 0)
    
TOTAL_ASSETS['Asset'] = asset_names
TOTAL_ASSETS.set_index(['Asset'], inplace=True, drop=True)

compact = ['1 Yr Z Score', '3 Yr Z Score','5 Yr Z Score', '1 Month Change', '3 Month Change']
TOTAL_ASSETS = TOTAL_ASSETS[compact]

## Display Results

In [54]:
th_props = [
  ('font-size', '16px'),
  ('text-align', 'center'),
  ('font-weight', 'bold'),
  ('color', 'Black'),
  ('background-color', '#f7f7f9')
  ]


# Set CSS properties for td elements in dataframe
td_props = [('font-size', '16px'), ('text-align', 'center')]

# Set table styles
styles = [
    dict(selector="th", props=th_props),
    dict(selector="td", props=td_props)
]

cm_low_good = sns.diverging_palette(150, 10, s=80, l=70, n=len(TOTAL_ASSETS.columns), as_cmap=True)#sns.light_palette("green", as_cmap=True)
cm_high_good = sns.diverging_palette(10, 150, s=80, l=70, n=len(TOTAL_ASSETS.columns), as_cmap=True)

TOTAL_ASSETS_VIEW = (TOTAL_ASSETS.style
    .applymap(color_negative_red, subset=['1 Yr Z Score', '3 Yr Z Score','5 Yr Z Score' ,'1 Month Change', '3 Month Change'])
    #.background_gradient(axis = 0 ,subset = ['DSSP', 'HIDI'],cmap=cm_high_good)
    #.background_gradient(axis = 0 ,subset = ['VRANK', 'MRANK','CRANK', 'ST'],cmap=cm_low_good)
    .set_caption('Speculator Positioning')
    .format({'1 Month Change': "{:.2f}",'3 Month Change': "{:.2f}", '1 Yr Z Score': "{:.2f}", '3 Yr Z Score': "{:.2f}",
            '5 Yr Z Score': "{:.2f}"})
    .set_table_styles(styles))

# Returns, Realized and Implied Volatility

In [47]:
#Prices

GOLD_price_tmp = yf.download('GLD')
SP500_price_tmp = yf.download('SPY')
NQ_price_tmp = yf.download('QQQ')
DXY_price_tmp = yf.download('DX-Y.NYB')
BIT_price_tmp = yf.download('BTC-USD')
TLT_price_tmp = yf.download('TLT')
USO_price_tmp = yf.download('USO')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [48]:
GOLD_price = pd.DataFrame(GOLD_price_tmp['Close'])
SP500_price = pd.DataFrame(SP500_price_tmp['Close'])
NQ_price = pd.DataFrame(NQ_price_tmp['Close'])
DXY_price = pd.DataFrame(DXY_price_tmp['Close'])
BIT_price = pd.DataFrame(BIT_price_tmp['Close'])
TLT_price = pd.DataFrame(TLT_price_tmp['Close'])
USO_price = pd.DataFrame(USO_price_tmp['Close'])


prices = [SP500_price,NQ_price,TLT_price,GOLD_price,USO_price , DXY_price, BIT_price  ]

price_names = ['SP500','QQQ','Bonds','Gold','Crude Oil', 'DXY', 'Bitcoin'  ]



In [49]:
imp_vol = pd.DataFrame()
imp_vol['Asset'] = price_names
imp_vol['ImpVol'] = np.nan 

tickers = ['SPY','QQQ' ,'TLT', 'GLD', 'USO', 'UUP']

month_delta = 1

for i in range(len(tickers)):
    try:
        price_ind = yf.download(tickers[i],start='2020-01-01', progress = False, threads = False)
        stock = yf.Ticker(tickers[i])
        option_dates = stock.options
        date = dt.datetime.now()
        date += relativedelta(months=month_delta) 
        expiry_one_month = min(option_dates, key=lambda x:abs(dt.datetime.strptime(x, "%Y-%M-%d")-date))
        current_price = float(price_ind['Close'].tail(1))
        
        calls = stock.option_chain(expiry_one_month).calls
        puts = stock.option_chain(expiry_one_month).puts
        
        #ATM Skew
        ATM_put = puts[puts['strike'] == min(puts['strike'], key=lambda x:abs(x-current_price))]
        ATM_call = calls[calls['strike'] == min(calls['strike'], key=lambda x:abs(x-current_price))]
        ATM_skew = float(ATM_put['impliedVolatility']) - float(ATM_call['impliedVolatility']) 
        
        imp_vol.loc[i,"ImpVol"] = float(ATM_put['impliedVolatility'])
        
    except:
        print("ERROR! INDEX: ", i, ', TICKER: ', tickers[i])



In [50]:
for price in prices:
    price['1 Month Return'] = price['Close'].pct_change(20)
    price['3 Month Return'] = price['Close'].pct_change(60)
    price['Volatility'] = np.log(price['Close']/price['Close'].shift()).dropna().ewm(span=60).std()*(252**0.5)
    price['3 year Volatility Z-Score'] = price['Volatility'].rolling(3 * 252).apply(lambda x: Z_score(x),raw=False) 


PRICES = None

for price in prices:
    
    PRICES = pd.concat([PRICES, price.tail(1)], axis = 0)
    
PRICES['Asset'] = price_names 
compact_1 = ['Asset', '1 Month Return', '3 Month Return', 'Volatility', '3 year Volatility Z-Score']
PRICES = PRICES[compact_1]
PRICES.set_index(['Asset'], inplace=True, drop=True)

PRICES = pd.merge(PRICES, imp_vol, how="left", on=['Asset'])
PRICES['Implied Vol Premium'] = PRICES['ImpVol'] - PRICES['Volatility']

In [51]:
th_props = [
  ('font-size', '16px'),
  ('text-align', 'center'),
  ('font-weight', 'bold'),
  ('color', 'Black'),
  ('background-color', '#f7f7f9')
  ]


# Set CSS properties for td elements in dataframe
td_props = [('font-size', '16px'), ('text-align', 'center')]

# Set table styles
styles = [
    dict(selector="th", props=th_props),
    dict(selector="td", props=td_props)
]


cm_low_good = sns.diverging_palette(150, 10, s=80, l=70, n=len(PRICES.columns), as_cmap=True)#sns.light_palette("green", as_cmap=True)
cm_high_good = sns.diverging_palette(10, 150, s=80, l=70, n=len(PRICES.columns), as_cmap=True)

PRICES_VIEW = (PRICES.style
    .applymap(color_negative_red, subset=['1 Month Return', '3 Month Return', 'Volatility', '3 year Volatility Z-Score', 'ImpVol', 'Implied Vol Premium'])
    .set_caption('Asset Metrics')
    .format({'1 Month Return': "{:.1%}",'3 Month Return': "{:.1%}", 'Volatility': "{:.1%}", '3 year Volatility Z-Score': "{:.2f}", 
            'ImpVol': "{:.1%}",'Implied Vol Premium': "{:.1%}" })
    .set_table_styles(styles))

## View metrics

In [52]:
PRICES_VIEW

,Asset,1 Month Return,3 Month Return,Volatility,3 year Volatility Z-Score,ImpVol,Implied Vol Premium
0,SP500,-2.5%,-3.1%,23.0%,0.20,21.7%,-1.3%
1,QQQ,-4.5%,-2.0%,30.3%,0.42,25.3%,-5.0%
2,Bonds,-3.8%,-2.6%,18.9%,0.35,18.9%,-0.1%
3,Gold,-2.1%,-6.9%,12.8%,-0.68,13.8%,1.0%
4,Crude Oil,-1.6%,-16.5%,42.6%,-0.11,45.1%,2.5%
5,DXY,2.4%,6.3%,8.3%,1.28,12.0%,3.6%
6,Bitcoin,-17.9%,2.1%,51.8%,-0.52,nan%,nan%


In [ ]:
TOTAL_ASSETS_VIEW

In [39]:
def AHL_signal(data, lookback = [63,42,21]):

    short  = data.ewm(span=lookback[0]).mean() - data.ewm(span=4*lookback[0]).mean()
    medium = data.ewm(span=lookback[1]).mean() - data.ewm(span=4*lookback[1]).mean()
    long = data.ewm(span=lookback[2]).mean() - data.ewm(span=4*lookback[2]).mean()

    #vol = data.pct_change(1).rolling(63).std()
    vol = data.rolling(63).std()

    short_scld = short/vol
    medium_scld = medium/vol
    long_scld = long/vol

    short_norm = short_scld/short_scld.rolling(252).std()
    medium_norm = medium_scld/medium_scld.rolling(252).std()
    long_norm = long_scld/long_scld.rolling(252).std()

    short_sig = (short_norm * np.exp((-short_norm**2)/4))/0.89
    medium_sig = (medium_norm * np.exp((-medium_norm**2)/4))/0.89
    long_sig = (long_norm * np.exp((-long_norm**2)/4))/0.89

    short_sig = short_sig.dropna()
    medium_sig = medium_sig.dropna()
    long_sig = long_sig.dropna()

    comb_signal = (short_sig + medium_sig + long_sig)/3
    
    return(comb_signal)

In [40]:
trend_df = pd.DataFrame()
tickers = ['GLD', 'SPY','EEM', 'TLT', 'DBC', 'DBA', 'USO', 'BTC-USD']

for tick in tickers:
 
    price_tmp = yf.download(tick)
    price_tmp = price_tmp['Close']
    price_df = pd.DataFrame(price_tmp)
    price_df.columns = [tick]
    trend_df = pd.concat([trend_df,price_df ], axis = 1)
    
    
trend_df = trend_df.dropna()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [41]:
lookback = [21,31,63]

vol_lookback = max(lookback)
vol = trend_df.rolling(vol_lookback).std()

look_back_dict = {}
df_signals = pd.DataFrame()
for look in lookback:
    trend_tmp = trend_df.ewm(span=look).mean() - trend_df.ewm(span=4*look).mean()
    scld_trend = trend_tmp/vol
    trend_norm = scld_trend/scld_trend.rolling(252).std()
    trend_sig = (trend_norm * np.exp((-trend_norm**2)/4))/0.89
    trend_sig = trend_sig.dropna()
    look_back_dict[look] = trend_sig
    df_signals = df_signals.add(trend_sig, fill_value=0)

df_signals = df_signals/len(lookback)

#correlation of signals

signal_corr = pd.DataFrame()
for look in lookback:
    signal_corr.loc[:,look] = look_back_dict[look].mean(axis = 1)


forcast_scalar = 10/0.55 #mean sbolute forcast
forecasts = df_signals *forcast_scalar
weights = np.ones(3)/3 #equal weight the systems 
DIV_factor = 1/np.sqrt(np.dot(np.dot(weights,signal_corr.corr()),weights))


CORR = forecasts.tail(25).corr()
corr_multiplyer = 1 #correlation of different systems
target_risk = 0.25
VOL = trend_df.pct_change().rolling(25).std() * np.sqrt(252)

position = (forecasts/10) *(target_risk/VOL) * 0.01 * DIV_factor#what instrument weights? 0.01 for now

In [42]:
position

,GLD,SPY,EEM,TLT,DBC,DBA,USO,BTC-USD
Date,,,,,,,,
2014-09-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-09-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-09-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-09-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-09-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2022-08-24,-0.038535,-0.000595,-0.014497,-0.017482,0.004775,-0.010543,0.001958,-0.007595
2022-08-25,-0.040913,-0.000130,-0.014539,-0.017466,0.005462,-0.010756,0.002295,-0.007496
2022-08-26,-0.038744,-0.000595,-0.014131,-0.017878,0.006342,-0.010420,0.002461,-0.007007


In [43]:
forecasts

,GLD,SPY,EEM,TLT,DBC,DBA,USO,BTC-USD
Date,,,,,,,,
2015-12-14,-15.305314,5.494299,-13.614671,-8.944978,-8.727638,-10.774249,-10.894298,16.402476
2015-12-15,-15.339967,5.104777,-13.702967,-9.088247,-9.069668,-10.839346,-11.125538,16.470386
2015-12-16,-15.454573,5.431502,-13.866445,-9.295319,-9.436586,-10.898436,-11.402130,16.534999
2015-12-17,-15.394103,5.055260,-13.968140,-8.388476,-9.876154,-10.970131,-11.796806,16.594353
2015-12-18,-15.461617,3.464595,-14.068339,-7.029592,-10.207174,-11.172900,-12.073938,16.650685
...,...,...,...,...,...,...,...,...
2022-08-24,-17.184633,-0.383389,-6.685523,-12.849199,3.363649,-6.467667,2.486453,-14.758131
2022-08-25,-17.177784,-0.085097,-7.371302,-12.654151,3.797910,-6.222681,2.865937,-14.605458
2022-08-26,-17.239410,-0.453957,-7.397328,-12.519041,4.280855,-5.766385,3.060005,-14.421302


In [21]:
df_signals.abs().mean()

GLD        0.563669
SPY        0.534014
EEM        0.589108
TLT        0.592341
DBC        0.585878
DBA        0.515718
USO        0.554366
BTC-USD    0.532639
dtype: float64